In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 준비
* InvoiceNo : 주문번호('C'로 시작하는 것은 취소주문)
* StockCode : 제품코드
* Description : 제품설명
* Quantity : 주문갯수
* InvoiceDate : 주문일자
* UnitPrice : 제품단가
* CustomerID : 고객번호
* Country : 주문고객의 국적

In [4]:
import openpyxl
df = pd.read_excel('data\\Online Retail.xlsx')
df.head()

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

# 데이터 정제
* 아래 데이터는 필터링한다.
    * CusomerID가 null인 데이터
    * 주문취소한 데이터
    * Quantity가 음수인 데이터
    * UnitPrice가 음수인 데이터


In [ ]:

df.copy?


In [5]:
df['CustomerID'].dtypes

NameError: name 'df' is not defined

In [6]:
# CustomerID가 null이 아닌 데이터
cond1 = df['CustomerID'].isnull()

# 주문 취소한 데이터
cond2 = df['InvoiceNo'].astype('str').str[0]=='C'

# Quantity가 음수인 데이터
cond3 = df['Quantity'] < 0

# UnitPrice가 음수인 데이터
cond4 = df['UnitPrice'] < 0

df = df.loc[ ~(cond1 | cond2 | cond3 | cond4)].copy()

df.shape

NameError: name 'df' is not defined

In [ ]:
df['Country'].value_counts()

In [7]:
# 대다수를 차지하는 영국 고객을 대상으로 한다.
cond = df['Country'] == 'United Kingdom'
df = df.loc[cond].copy()

df['Country'].value_counts()

NameError: name 'df' is not defined

In [ ]:
# Code성 데이터는 보통 str
# CustomerID 자료형 : float -> int -> str
df['CustomerID'] = df['CustomerID'].astype('int').astype('str')

df.dtypes

# 데이터 탐색

In [ ]:
# 하나의 주문번호에 여러 주문 건수가 포함되어 있음
df['InvoiceNo'].value_counts()

In [ ]:
cond = df['InvoiceNo']==576339
df.loc[cond]

In [ ]:
cond = df['InvoiceNo']==555270
df.loc[cond]

# 데이터 가공
* RFM 분석을 위한 정보 생성

In [ ]:
# 주문 금액 데이터 생성 (Quantity * Unitpirce)
df['Amount'] = df['Quantity'] * df['UnitPrice']
df.head()

In [ ]:
# R : 고객별 가장 최근 날짜
# F : 고객별 중복 제외한 InvoceNo 갯수
# M : 고객별 Price 합계

aggregation = {
    'InvoiceDate' : 'max',
    'InvoiceNo' : 'nunique',
    'Amount' : 'sum'
}

df_rfm = df.groupby('CustomerID').agg(aggregation)
df_rfm.head(3)

In [ ]:
# 최근날짜
df_rfm['InvoiceDate'].max()

In [ ]:
df_rfm['InvoiceDate'].max()+pd.Timedelta(days=1)

In [ ]:
# InvoceDate : 현재부터 경과한 날짜
# 2011-12-10일 기준으로 경과한 날짜 계산

# 기준일 : 마지막날짜+1일
today = df_rfm['InvoiceDate'].max()+pd.Timedelta(days=1)

# 기준일시 - 주문일시
df_rfm['Recency'] = today - df_rfm['InvoiceDate']

# 날짜만 추출
df_rfm['Recency'] = df_rfm['Recency'].dt.days

df_rfm.head()

In [ ]:
# 필요한 데이터만 선택
df_rfm = df_rfm.loc[:,['Recency','InvoiceNo','Amount']].copy()
df_rfm.head()

In [ ]:
# 컬럼명 변경
df_rfm = df_rfm.rename(columns={
              'InvoiceNo':'Frequency',
              'Amount':'Monetary'
              })

df_rfm.head()

In [ ]:
plt.figure(figsize=(15,4))
for i, col in enumerate(df_rfm.columns):
    plt.subplot(1,3,i+1)
    sns.histplot(data=df_rfm, x=col)
    plt.title(col)

In [ ]:
# 데이터 로그변환
df_rfm['Recency_log'] = np.log1p(df_rfm['Recency'])
df_rfm['Frequency_log'] = np.log1p(df_rfm['Frequency'])
df_rfm['Monetary_log'] = np.log1p(df_rfm['Monetary'])

df_rfm.head()

In [ ]:
# 스케일링 대상 : 로그변환 한 데이터
X = df_rfm.loc[:,'Recency_log':'Monetary_log']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # --> 세그먼테이션에 사용할 데이터
X_scaled

# 고객 세그먼테이션

In [ ]:
# 엘보우 기법으로 최적의 k 찾기
from sklearn.cluster import KMeans
inertia = []
for n in range(1,10):
    km = KMeans(n_clusters=n)
    km.fit(X_scaled)
    print(km.inertia_)
    inertia.append(km.inertia_)

In [ ]:
plt.plot(range(1,10), inertia, marker='o')
plt.xticks(range(1,10))
plt.xlabel('k')
plt.ylabel('inertia')
plt.show()

In [ ]:
# 최적의 실루엣 찾기
import silhouette_analysis as s
for k in range(2,6):
    s.silhouette_plot(X_scaled, k)

In [ ]:
k = 2
from sklearn.cluster import KMeans
km = KMeans(n_clusters=k)
df_rfm['cluster'] = km.fit_predict(X_scaled)
df_rfm.head()

In [ ]:
d = df_rfm[['Recency_log','Frequency_log','Monetary_log','cluster']]
sns.pairplot(d, hue='cluster', palette='muted')

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(data=df_rfm, x='Recency_log', y='Monetary_log', 
                hue='cluster', alpha=0.5, palette='muted')

In [112]:
cond0= df_rfm['cluster']==0
cond1= df_rfm['cluster']==1
cond2= df_rfm['cluster']==2
cond3= df_rfm['cluster']==3


df0 = df_rfm.loc[cond0]
df1 = df_rfm.loc[cond1]
df2 = df_rfm.loc[cond2]
df3 = df_rfm.loc[cond3]


dfs = [df0, df1]
dfs = [df0, df1, df2]
dfs = [df0, df1, df2, df3]


In [ ]:
plt.figure(figsize=(15,3))
for i, d in enumerate(dfs):
    plt.subplot(1,k,i+1)
    sns.violinplot(data=d, y='Recency')
    plt.ylim(0,400)
plt.tight_layout()
plt.show()

plt.figure(figsize=(15,3))
for i, d in enumerate(dfs):
    plt.subplot(1,k,i+1)
    sns.violinplot(data=d, y='Frequency')
    plt.ylim(0,200)
plt.tight_layout()
plt.show()

plt.figure(figsize=(15,3))
for i, d in enumerate(dfs):
    plt.subplot(1,k,i+1)
    sns.violinplot(data=d, y='Monetary')
    plt.ylim(0,25000)
plt.tight_layout()
plt.show()

In [ ]:

plt.figure(figsize=(10,4))

for i, df in enumerate(dfs):
    plt.subplot(1,k,i+1)
    sns.scatterplot(data=df, x='Recency', y='Monetary', 
                hue='cluster', alpha=0.5, palette='muted')
    plt.xlim(0,400)
    plt.ylim(0,25000)

plt.tight_layout()



In [ ]:

plt.figure(figsize=(10,4))

for i, df in enumerate(dfs):
    plt.subplot(1,k,i+1)
    sns.scatterplot(data=df, x='Frequency', y='Monetary', 
                hue='cluster', alpha=0.5, palette='muted')
    plt.xlim(0,150)
    plt.ylim(0,25000)

plt.tight_layout()



In [ ]:

plt.figure(figsize=(10,4))

for i, df in enumerate(dfs):
    plt.subplot(1,k,i+1)
    sns.scatterplot(data=df, x='Recency', y='Frequency', 
                hue='cluster', alpha=0.5, palette='muted')
    plt.xlim(0,400)
    plt.ylim(0,200)

plt.tight_layout()



In [ ]:
df_rfm[['Recency', 'Frequency', 'Monetary', 'cluster']]

In [ ]:
cols = ['Recency', 'Frequency', 'Monetary']
plt.figure(figsize=(15,4))
for i, col in enumerate(cols):
    plt.subplot(1,3,i+1)
    sns.violinplot(data=df_rfm, y=col, hue='cluster',palette='muted')
    plt.title(col)

    # 0번군집의 특성 : 최근/오래된 고객 통틀어서 구입빈도가 낮고 구입금액이 낮은 고객
    # 1번 군집의 특성 : 최근에 들어온 편, 자주 구매하는 편, 구매금액 괜찮은편
    # 2번 군집의 특성 : 
    # 3번 군집의 특성 : 